In [ ]:
# %pip install bingmaps

In [1]:
import bingmaps
from bingmaps.apiservices import LocationByQuery
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)
tqdm.pandas()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
pd.__version__

'2.0.0rc0'

In [2]:
df = pd.read_csv('7_address.csv')
pd.options.display.max_columns = 100
df

/var/folders/1p/747b_4cn70v5h3223zdf0k180000gp/T/ipykernel_9542/3739346949.py:1: DtypeWarning: Columns (56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('7_address.csv')


,Unnamed: 0,make_model,short_description,make,model,location,price,body_type,type,doors,country_version,offer_number,warranty,mileage,first_registration,gearbox,fuel_type,colour,paint,desc,seller,seats,power,engine_size,gears,co2_emissions,manufacturer_colour,drivetrain,cylinders,fuel_consumption,comfort_convenience,entertainment_media,safety_security,extras,empty_weight,model_code,general_inspection,last_service,full_service_history,non_smoker_vehicle,emission_class,emissions_sticker,upholstery_colour,upholstery,production_date,previous_owner,other_fuel_types,power_consumption,energy_efficiency_class,co2_efficiency,fuel_consumption_(wltp),co2_emissions_(wltp),available_from,taxi_or_rental_car,availability,last_timing_belt_change,electric_range_(wltp),power_consumption_(wltp),battery_ownership,desc_lang,desc_lang2,desc_en,address,latitude,longitude
0,0,Mercedes-Benz A 160,CDi,Mercedes-Benz,A 160,"P.I. EL PALMAR C/FORJA 6, 11500 PUERTO DE SAN...","€ 16,950.-","['\n, Compact, \n']","['\n, Used, \n']","['\n, 5, \n']","['\n, Spain, \n']","['\n, 7468157, \n']","['\n, 24 months, \n']","120,200 km",06/2016,['\nManual\n'],Diesel,Black,Metallic,"['Precio al contado: 16950 euros', 'Vehículo c...",Dealer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 16950 euros',' vehicle with main...","Calle Forja 6, 11500 El Puerto de Santa María,...",36.618987,-6.205012
1,1,Mercedes-Benz EQE 350,350+,Mercedes-Benz,EQE 350,"APARTADO DE CORREOS 1032, 26140 LOGROÑO, ES","€ 80,900.-","['\n, Compact, \n']","['\n, Pre-registered, \n']","['\n, 4, \n']","['\n, Spain, \n']","['\n, 7054863, \n']","['\n, 24 months, \n']","5,000 km",06/2022,['\nAutomatic\n'],Electric,NaN,NaN,"['Precio al contado: 88900 euros', 'AUTO OJA S...",Dealer,"['\n, 5, \n']",['\n215 kW (292 hp)\n'],['\n1 cc\n'],['\n9\n'],0 g/km (comb.),Negro Obsidiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 88900 euros',' Auto Oja S.A. - O...","Calle Logroño, 28649 Rozas de Puerto Real, Mad...",40.315198,-4.474838
2,2,Mercedes-Benz A 45 AMG,S 4Matic+ 8G-DCT,Mercedes-Benz,A 45 AMG,"PORT. TARRACO, MOLL DE LLEVANT, Nº 5, LOC. 6-8...","€ 69,900.-","['\n, Compact, \n']","['\n, Used, \n']","['\n, 5, \n']","['\n, Spain, \n']","['\n, 7410454, \n']","['\n, 12 months, \n']","18,900 km",07/2020,['\nAutomatic\n'],Gasoline,Grey,NaN,"['Gris MANUFAKTUR mountaingrau magno', 'Linea/...",Dealer,"['\n, 5, \n']",['\n310 kW (421 hp)\n'],"['\n1,991 cc\n']",['\n8\n'],NaN,Mountaingrau magno,"['\n, 4WD, \n']",['\n4\n'],"[['8.4 l/100 km (comb.)'], ['10.9 l/100 km (ci...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Gray Manufakur Mountingrau Magno', 'Line/Sty...","Muelle de Llevant, 43004 Tarragona, Spain",41.106726,1.249235
3,3,Mercedes-Benz A 35 AMG,4Matic+ 7G-DCT,Mercedes-Benz,A 35 AMG,"Carrer de Provença, 31 Local, 8029 BARCELONA, ES","€ 46,990.-","['\n, Compact, \n']","['\n, Used, \n']","['\n, 5, \n']","['\n, Spain, \n']","['\n, 7464503, \n']","['\n, 12 months, \n']","18,995 km",01/2020,['\nAutomatic\n'],Gasoline,Yellow,NaN,"['Precio al contado: 48990 euros', 'BONIFICACI...",Dealer,"['\n, 5, \n']",['\n225 kW (306 hp)\n'],"['\n1,991 cc\n']",['\n7\n'],NaN,Amarillo Sol,"['\n, 4WD, \n']",['\n4\n'],"[['7.3 l/100 km (comb.)'], ['9.3 l/100 km (cit...","['Air conditioning, Automatic climate control,...",['USB'],"['ABS, Central door lock, Driver-side airbag, ...",['Sport seats'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,es,"['Cash price: 48990 euros',' Bonus to finance:...","Calle Provenza, Valle de los Molinos, 45200 Gu...",20.848540,-103.428946
4,4,Mercedes-Benz A 45 AMG,200CDI BE Line 4M 7G-DCT,Mercedes-Benz,A 45 AMG,"CARRIL ARAGONES 4, 30007 CASILLAS, ES","€ 16,

In [ ]:
df.location

In [ ]:
df.location.isna().value_counts()

In [ ]:
df.location[0]

In [3]:
# key -> https://www.bingmapsportal.com/Application
bingmaps_key = 'AgRrM5clyPtcsKJkfg4ab4Q8P4UeQFq9mP_IJs97Hxb1wB1OdL3bBaedqN6x2dMT'
# define the address to correct
def getCorrectedAddressAndCoordinates(x):
    try:
        address = x

        # set up the query parameters
        params = {
            'q': address,
            'key': bingmaps_key,
        }

        # use the geocode API to get the corrected address
        response = LocationByQuery(params)

        # print the corrected address
        if response.status_code == 200:
            return response.get_address[0]["formattedAddress"],response.get_coordinates[0][0], response.get_coordinates[0][1]
        else:
            return address,np.nan,np.nan
    except:
        return x,np.nan,np.nan
getCorrectedAddressAndCoordinates(df.location[0])

('Calle Forja 6, 11500 El Puerto de Santa María, Cadiz, Spain',
 36.6189866,
 -6.2050122)

In [7]:
getCorrectedAddressAndCoordinates('AVDA.DELS ARCS,20 POL. IND. AEROPUERTO,  46940 MANISES, ES')

('AVDA.DELS ARCS,20 POL. IND. AEROPUERTO,  46940 MANISES, ES', nan, nan)

In [ ]:
# DONE
# df['location_corrected'] = df.location.parallel_apply(lambda x: getCorrectedAddressAndCoordinates(x))

In [ ]:
# DONE
# df['address'] = df.location_corrected.apply(lambda x: x[0])
# df['latitude'] = df.location_corrected.apply(lambda x: x[1])
# df['longtitude'] = df.location_corrected.apply(lambda x: x[2])
# df.drop(columns=['location_corrected'], inplace = True)

In [8]:
df.address

0        Calle Forja 6, 11500 El Puerto de Santa María,...
1        Calle Logroño, 28649 Rozas de Puerto Real, Mad...
2                Muelle de Llevant, 43004 Tarragona, Spain
3        Calle Provenza, Valle de los Molinos, 45200 Gu...
4            Carril de los Aragonés 4, 30007 Murcia, Spain
                               ...                        
28625    Calle de los Hermanos Lumière 16, 29004 Málaga...
28626            Stobbeweg 6, 2461 EX Ter Aar, Netherlands
28627                                        5060, Belgium
28628    Bergambachterstraat 5, 2871 JB Schoonhoven, Ne...
28629            Lage Ham 106, 5102 AE Dongen, Netherlands
Name: address, Length: 28630, dtype: object

In [ ]:
# pd.options.display.max_rows = 100

In [8]:
def getCountry(x):
    try:
        address = x

        # set up the query parameters
        params = {
            'q': address,
            'key': bingmaps_key,
        }

        # use the geocode API to get the corrected address
        response = LocationByQuery(params)

        # print the corrected address
        if response.status_code == 200:
            data = json.loads(response.response)
            try:
                country_region = data['resourceSets'][0]['resources'][0]['address']['countryRegion']
                return country_region
            except:
                return np.nan
        else:
            return np.nan
    except:
        return np.nan

In [9]:
getCountry(df.iloc[6301,:].address)

'Netherlands'

In [34]:
df['country'] = df.location.parallel_apply(lambda x: getCountry(x))

In [37]:
df.to_csv('8_country.csv')